In [1]:
using InteractiveUtils, DrWatson, Comonicon
if isdefined(Main, :IJulia) && Main.IJulia.inited
    using Revise
else
    ENV["GKSwstype"] = 100 # suppress warnings during gif saving
end
versioninfo()
@quickactivate

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 10


In [3]:
using Plots, ProgressMeter, Logging
theme(:bright; size=(300, 300))

In [4]:
using Random, Turing, BayesianSymbolic
using ExprOptimization.ExprRules
using PyCall
pd = pyimport("pandas")

includef(args...) = isdefined(Main, :Revise) ? includet(args...) : include(args...)
includef(srcdir("utility.jl"))
includef(srcdir("app_inf.jl"))
includef(srcdir("sym_reg.jl"))
includef(srcdir("exp_max.jl"))
includef(srcdir("analyse.jl"))
includef(srcdir("dataset.jl"))
# Suppress warnings of using _varinfo
with_logger(SimpleLogger(stderr, Logging.Error)) do
    includef(srcdir("scenarios", "magnet.jl"))
    includef(srcdir("scenarios", "nbody.jl"))
    includef(srcdir("scenarios", "bounce.jl"))
    includef(srcdir("scenarios", "mat.jl"))
end

┌ Info: Precompiling BayesianSymbolic [d95aa7d0-ea3e-4103-b443-9ed45b862455]
└ @ Base loading.jl:1317
┌ Info: Skipping precompilation since __precompile__(false). Importing BayesianSymbolic [d95aa7d0-ea3e-4103-b443-9ed45b862455].
└ @ Base loading.jl:1025
┌ Warning: skipping callee #_#38 (called by nothing) due to UndefRefError()
└ @ LoweredCodeUtils /afs/inf.ed.ac.uk/user/s16/s1672897/.julia/packages/LoweredCodeUtils/poBmh/src/signatures.jl:279


In [5]:
c00 = count_expressions(G_BSP_00, 5, :Force)
c01 = count_expressions(G_BSP_01, 5, :Force)
c10 = count_expressions(G_BSP_10, 5, :Force)
c11 = count_expressions(G_BSP_11, 5, :Force)
@info "Depth 5" c00 c01 c10 c11

c11 = count_expressions(G_BSP_11, 6, :Force)
@info "Depth 6" c11

┌ Info: Depth 5
│   c00 = 8593200
│   c01 = 7935408
│   c10 = 97929
│   c11 = 75816
└ @ Main In[5]:5
┌ Info: Depth 6
│   c11 = 771120
└ @ Main In[5]:8


In [38]:
function collect_results(dataset, method; verbose=false, priordim=true, priortrans=true)
    dataset = "synth/$dataset"
    df = pd.DataFrame(
        columns=("nRMSE", "#Scenes", "Seed", "Seed (Data)")
    )

    count = 1
    @showprogress for ntrains in 1:10, seed in 1:5, shuffleseed in 1:5
        hps = (ntrains=ntrains, seed=seed, shuffleseed=shuffleseed, priordim=priordim, priortrans=priortrans)

        dir = datadir(dataset)
        scenarios, attributes = loaddata(dir, hps.ntrains; shuffleseed=hps.shuffleseed)
        scenarios_test, attributes_test = loaddata(dir, 20; shuffleseed=hps.shuffleseed, istrain=false)
        attributes = [missing]
        attributes_test = [missing]

        local res
        try
            res = wload(resultsdir(dataset, savename(hps; connector="-"), "monly.jld2"))
        catch
            println("seed $(hps.seed) unfinished")
            continue
        end

        @unpack ScenarioModel, latentname, ealg, malg, elike, mlike, force = res
        
        if method == "Zero"
            force = ZeroForce()
        end

        verbose && println("count: $count")
        verbose && if method == "BSP"
            println(BayesianSymbolic.get_executable(force.tree, force.grammar))
            println(force.constant)
        end
        evalres = evalforce(ScenarioModel, scenarios, attributes, force, ealg, elike, mlike)
        evalres_test = evalforce(ScenarioModel, scenarios_test, attributes_test, force, ealg, elike, mlike)

        # Add to data frame
        df.loc[count] = [evalres_test.normrmse, ntrains, seed, shuffleseed]
        count = count + 1
        verbose && println()
    end
    
    return df
end

collect_results (generic function with 1 method)

In [ ]:
df = collect_results("nbody", "BSP")
# df11 = collect_results("nbody", "BSP", priordim=true, priortrans=true)

In [ ]:
df.to_csv("pretty_paper/raw/monly-nbody-symbolic.csv", header=false)

df11 = df
df11.to_csv("pretty_paper/raw/monly-nbody-g11.csv", header=false)

In [39]:
df01 = collect_results("nbody", "BSP", priordim=false, priortrans=true)

Progress:  50%|████████████████████▌                    |  ETA: 0:22:16

seed 4 unfinished


Progress: 100%|█████████████████████████████████████████| Time: 0:49:03


,nRMSE,#Scenes,Seed,Seed (Data)
0,0.729807,1.0,1.0,1.0
1,1.390339,1.0,1.0,2.0
2,1.442338,1.0,1.0,3.0
3,1.399944,1.0,1.0,4.0
4,1.590519,1.0,1.0,5.0
...,...,...,...,...
244,1.368318,10.0,5.0,1.0
245,1.319723,10.0,5.0,2.0
246,1.330186,10.0,5.0,3.0
247,1.300984,10.0,5.0,4.0


In [40]:
df01.to_csv("pretty_paper/raw/monly-nbody-g01.csv", header=false)

In [41]:
df10 = collect_results("nbody", "BSP", priordim=true, priortrans=false)

Progress: 100%|█████████████████████████████████████████| Time: 0:58:33


,nRMSE,#Scenes,Seed,Seed (Data)
0,1.095121,1.0,1.0,1.0
1,0.922785,1.0,1.0,2.0
2,0.931279,1.0,1.0,3.0
3,0.983847,1.0,1.0,4.0
4,1.144681,1.0,1.0,5.0
...,...,...,...,...
245,0.551785,10.0,5.0,1.0
246,1.341670,10.0,5.0,2.0
247,1.351490,10.0,5.0,3.0
248,0.659961,10.0,5.0,4.0


In [42]:
df10.to_csv("pretty_paper/raw/monly-nbody-g10.csv", header=false)

In [43]:
df00 = collect_results("nbody", "BSP", priordim=false, priortrans=false)

Progress: 100%|█████████████████████████████████████████| Time: 0:46:10


,nRMSE,#Scenes,Seed,Seed (Data)
0,1.787204,1.0,1.0,1.0
1,1.196310,1.0,1.0,2.0
2,0.799505,1.0,1.0,3.0
3,1.286916,1.0,1.0,4.0
4,1.316368,1.0,1.0,5.0
...,...,...,...,...
245,1.383326,10.0,5.0,1.0
246,1.342126,10.0,5.0,2.0
247,1.351615,10.0,5.0,3.0
248,1.330902,10.0,5.0,4.0


In [44]:
df00.to_csv("pretty_paper/raw/monly-nbody-g00.csv", header=false)

## Deprecated

In [34]:
using BSON, DelimitedFiles

dirprefix = "/afs/inf.ed.ac.uk/user/s16/s1672897/projects/bayesian_symbolic_physics/data/archived/data_effiency"

for (dataset_in, dataset_out) in [("NBODY/stable", "nbody"), ("BOUNCE/4-body", "bounce"), ("MAT/1-mat", "mat")]
    for method in ["in", "mlpdynamics", "mlpforce", "ogn", "symbolic", "naivegrammar", "naivegrammar2"]
        data = []
        for n_data in 1:10, seed in 1:10
            try
                d = BSON.load("$dirprefix/$dataset_in/$method/n_data=$n_data-seed=$seed.bson")
                push!(data, [d[:vec], n_data, -1, seed])
            catch
                @info "Data not existing" dataset_out method n_data seed
            end
        end
        writedlm("pretty_paper/raw/monly-$dataset_out-$method-old.csv", data, ',')
    end
end

┌ Info: Data not existing
│   dataset_out = nbody
│   method = mlpdynamics
│   n_data = 10
│   seed = 10
└ @ Main In[34]:13
┌ Info: Data not existing
│   dataset_out = bounce
│   method = mlpdynamics
│   n_data = 10
│   seed = 10
└ @ Main In[34]:13
┌ Info: Data not existing
│   dataset_out = mat
│   method = in
│   n_data = 10
│   seed = 10
└ @ Main In[34]:13
┌ Info: Data not existing
│   dataset_out = mat
│   method = mlpdynamics
│   n_data = 10
│   seed = 10
└ @ Main In[34]:13
┌ Info: Data not existing
│   dataset_out = mat
│   method = mlpforce
│   n_data = 10
│   seed = 10
└ @ Main In[34]:13
